# SQL Analysis For A Well Known Lender

Right before i joined my current company i completed a fun SQL test for a well know Lender in the UK. This section should help anyone looking to work with SQLite in Python. Keep reading to find out when most people default on their loans. 

Key Demonstation: Loading and cleaning a database in Python and SQL queries. 

In [1]:
import numpy as np
import pandas as pd
import sqlite3

In [6]:
# Establish DB connection and clean
conn = sqlite3.connect('example_loan_data.db')

c = conn.cursor()

conn.execute('VACUUM')
conn.commit()


In [20]:
# verify count and schemas
sqlite_query = """
SELECT COUNT(*)
FROM loan_book
"""
sqlite_query2 = """
SELECT name from sqlite_master where type= "table"
"""
sqlite_query3 = """
SELECT * 
from loan_book 
LIMIT 1000
"""
sqlite_query4 = """
SELECT * 
from default_rate_estimate
"""

print(pd.read_sql(sqlite_query, conn))
print(pd.read_sql(sqlite_query2, conn))
print(pd.read_sql(sqlite_query3, conn).head())
print(pd.read_sql(sqlite_query4, conn).head())

   COUNT(*)
0     83489
                    name
0              loan_book
1  default_rate_estimate
  market   Amount  Term_nMonths  Annual_interest_rate  \
0      B  10000.0            60                0.0997   
1     A1   7500.0            48                0.0302   
2     A2  15000.0            60                0.0790   
3     A2  10000.0            48                0.0647   
4     C1   5000.0            36                0.0946   

            Date_of_disbursal Date_of_default  Declared_income  \
0  2016-04-04 09:20:04.903000            None          66318.0   
1  2016-04-06 12:00:43.360000            None          36000.0   
2  2016-04-05 11:12:03.257000            None          35000.0   
3  2016-10-31 11:04:04.047000            None          37626.0   
4  2016-11-01 16:38:58.930000            None          23460.0   

               loan_id            member_id  
0  6291910946835746176  5952682155246103118  
1 -2243521942468637182 -3855500498482240246  
2  4840418347660553669 

## 1. How many loans are there in each market?

In [9]:
sqlite_query = """
SELECT market AS Unique_markets, COUNT(market) AS Loans_per_market
FROM loan_book
GROUP BY market
"""

pd.read_sql(sqlite_query, conn)

,Unique_markets,Loans_per_market
0,A*,18963
1,A1,11069
2,A2,11249
3,B,11878
4,C1,14133
5,D,9810
6,E,6368


## 2. Find the top 3 calendar months on the loan book that have highest average values of estimated annual default rate

Looks like most people default on loans at Christmas. 

In [10]:
sqlite_query = """
SELECT strftime('%m', loan_book.Date_of_disbursal) AS Calender_Month, AVG(default_rate_estimate.Annual_default_rate_estimate) AS Average_Annual_Default_Rate
FROM loan_book
INNER JOIN default_rate_estimate
ON default_rate_estimate.market = loan_book.market and default_rate_estimate.Term_nMonths = loan_book.Term_nMonths
GROUP BY strftime('%m', loan_book.Date_of_disbursal)
ORDER BY Average_Annual_Default_Rate DESC
LIMIT 3
"""

pd.read_sql(sqlite_query, conn)

,Calender_Month,Average_Annual_Default_Rate
0,12,0.061691
1,11,0.060038
2,09,0.056131


## 3. Based on the loan book, what is the average of estimated annual default rate in each market? Compare this with the actual default rate within the first year from disbursal using the loan book.

Hint: the date_of_default column indicates whether the loan has actually defaulted and when it happened

In [11]:
sqlite_query = """
/* Find average estimated annual defualt rate */
SELECT loan_book.market AS Market, AVG(default_rate_estimate.Annual_default_rate_estimate) AS AVG_Estimated_Annual_Defualt, sum(a.Total_defualts_within_1year +0.00000) / sum(b.Total_Loans2016 +0.00000) AS Actual_Default_Rate_Within_First_Year
FROM loan_book

INNER JOIN default_rate_estimate
ON default_rate_estimate.market = loan_book.market and default_rate_estimate.Term_nMonths = loan_book.Term_nMonths

INNER JOIN

/*Use julianday to find the number of defualts within 365 days of disbursal */
(SELECT COUNT(Date_of_default) AS Total_defualts_within_1year, market
FROM loan_book
WHERE julianday(Date_of_default) <= julianday(Date_of_disbursal) + 365
GROUP BY market) as a
ON a.market = loan_book.market

INNER JOIN

/*Find total loans in 2016 */
(SELECT COUNT(Date_of_disbursal) AS Total_Loans2016, market
FROM loan_book
GROUP BY market) as b
ON b.market = loan_book.market

GROUP BY loan_book.market
"""

pd.read_sql(sqlite_query, conn)




,Market,AVG_Estimated_Annual_Defualt,Actual_Default_Rate_Within_First_Year
0,A*,0.003761,0.000896
1,A1,0.011284,0.004156
2,A2,0.021466,0.008801
3,B,0.041078,0.019027
4,C1,0.071021,0.037006
5,D,0.145736,0.060143
6,E,0.152952,0.096891


## 4. How many of these repeat borrowers got a new loan with a lower interest rate than their previous loan?

In [13]:

sqlite_queryA = """
SELECT member_id, Annual_interest_rate, Date_of_disbursal
FROM loan_book
WHERE member_id IN 
(SELECT member_id
FROM loan_book
GROUP BY member_id
HAVING COUNT(*) > 1)
ORDER BY member_id, Date_of_disbursal DESC

"""

A = pd.read_sql(sqlite_queryA, conn)
NEW = A.groupby("member_id").first() #Selects newest interest rate
OLD = A.groupby("member_id").nth(1) #Selects previous interest rate
mask = NEW["Annual_interest_rate"].values < OLD["Annual_interest_rate"].values #Boolean array with lower interest rates = TRUE
print(sum(mask))

841
